gensim是可以计算句向量的，相关模块是models.doc2vec – Deep learning with paragraph2vec

这种求句向量的方法是基于google的2014年的paper，也就是paragraph2vec方法。这种方法和word2vec是非常类似的，比如用skip-gram，就是在word2vec从中心词预测周围词时，不仅考虑中心词和周围词的词向量，在计算损失函数时还要考虑中心词所在document的代表性的vector，这个vector和中心词和周围词的词向量一起，作为最初的输入。拟合完成后，所有的出现过的document就都有了对应的vector，那么，如果给出一个新的document，如何知道它的vector呢？其实是把新的document和其中的词放入模型中重新去拟合更新，得到新的document的vector。

从上面的过程看出，paragraph2vec的方法有点naive，也许都不一定比直接使用word2vec取平均更好（需要用句向量任务验证下），和2018年google发布的universal sentence encoder更是没法比。但既然相关方法已经在gensim实现了，不妨一试，反正不是很费力气。

[Doc2Vec Tutorial on the Lee Dataset](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)

[models.doc2vec – Deep learning with paragraph2vec](https://radimrehurek.com/gensim/models/doc2vec.html)


In [ ]:
import gensim
import os
import collections
import smart_open
import random

In [ ]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

In [ ]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [ ]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))


In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=55)

In [ ]:
model.build_vocab(train_corpus)

In [ ]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])